In [25]:
import polars as pl
from scipy.spatial.distance import pdist, squareform
import numpy as np
from tqdm import tqdm
import os
import cv2




In [4]:
df = pl.scan_csv('/home/dangnh36/datasets/.comp/byu/processed/gt_v2.csv').with_columns(
    pl.col('motor_zyxA', 'motor_zyx').map_elements(eval, return_dtype = pl.Object)
).collect()
df

tomo_id,Z,Y,X,voxel_spacing,ori_num_motors,num_motors,ZA,YA,XA,V,VA,motor_z,motor_y,motor_x,motor_zA,motor_yA,motor_xA,motor_zyx,motor_zyxA
str,i64,i64,i64,f64,i64,i64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,f64,object,object
"""tomo_003acc""",500,1912,1847,6.5,0,0,3250.0,12428.0,12005.5,1765732000,4.8491e11,-1.0,-1.0,-1.0,-6.5,-6.5,-6.5,[],[]
"""tomo_00e047""",300,959,928,15.6,1,1,4680.0,14960.4,14476.8,266985600,1.0136e12,169.0,546.0,603.0,2636.4,8517.6,9406.8,"[[169.0, 546.0, 603.0]]","[[2636.4, 8517.6, 9406.8]]"
"""tomo_00e463""",500,924,956,19.7,6,6,9850.0,18202.8,18833.2,441672000,3.3767e12,235.0,403.0,137.0,4629.5,7939.1,2698.9,"[[235.0, 403.0, 137.0], [243.0, 363.0, 153.0], [222.0, 379.0, 144.0], [225.0, 262.0, 628.0], [225.0, 241.0, 643.0], [231.0, 289.0, 632.0]]","[[4629.5, 7939.099999999999, 2698.9], [4787.099999999999, 7151.099999999999, 3014.1], [4373.4, 7466.3, 2836.7999999999997], [4432.5, 5161.4, 12371.6], [4432.5, 4747.7, 12667.1], [4550.7, 5693.3, 12450.4]]"
"""tomo_01a877""",300,960,928,13.1,1,1,3930.0,12576.0,12156.8,267264000,6.0083e11,147.0,638.0,286.0,1925.7,8357.8,3746.6,"[[147.0, 638.0, 286.0]]","[[1925.7, 8357.8, 3746.6]]"
"""tomo_02862f""",300,959,928,15.6,1,1,4680.0,14960.4,14476.8,266985600,1.0136e12,101.0,351.0,120.0,1575.6,5475.6,1872.0,"[[101.0, 351.0, 120.0]]","[[1575.6, 5475.599999999999, 1872.0]]"
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""tomo_fe050c""",300,959,928,15.6,1,2,4680.0,14960.4,14476.8,266985600,1.0136e12,138.0,134.0,542.0,2152.8,2090.4,8455.2,"[[138.0, 134.0, 542.0], [143.125, 93.875, 405.75]]","[[2152.7999999999997, 2090.4, 8455.199999999999], [2232.75, 1464.45, 6329.7]]"
"""tomo_fe85f6""",800,928,960,13.1,0,1,10480.0,12156.8,12576.0,712704000,1.6022e12,463.5,903.5,747.0,6071.85,11835.85,9785.7,"[[463.5, 903.5, 747.0]]","[[6071.849999999999, 11835.85, 9785.699999999999]]"
"""tomo_fea6e8""",300,960,928,16.8,0,0,5040.0,16128.0,15590.4,267264000,1.2673e12,-1.0,-1.0,-1.0,-16.8,-16.8,-16.8,[],[]


In [27]:
SAVE_DIR = '/home/dangnh36/datasets/.comp/byu/processed/viz/annotations/'
os.makedirs(SAVE_DIR, exist_ok= True)

for i, row in tqdm(enumerate(df.iter_rows(named = True))):
    tomo_id = row['tomo_id']
    ori_spacing = row['voxel_spacing']
    Z, Y, X = row['Z'], row['Y'], row['X']
    tomo_dir = f'/home/dangnh36/datasets/.comp/byu/raw/train/{tomo_id}'
    save_path = os.path.join(SAVE_DIR, f'{tomo_id}.jpg')
    motor_zyxs = row['motor_zyx']
    if len(motor_zyxs) == 0:
        select_z = Z // 2
        img = cv2.imread(os.path.join(tomo_dir, f'slice_{select_z:04d}.jpg'))
        cv2.imwrite(save_path, img)
        continue

    vizs = []
    for motor_zyx in motor_zyxs:
        mz, my, mx = [round(e) for e in motor_zyx]
        img = cv2.imread(os.path.join(tomo_dir, f'slice_{mz:04d}.jpg'))
        radius = round(1000 / ori_spacing)
        cv2.circle(img, (mx, my), radius = radius, color = (0, 255, 0), thickness = 3)
        vizs.append(img)
    viz = np.concatenate(vizs, axis = 1)
    cv2.imwrite(save_path, viz)

648it [00:17, 36.83it/s]


In [33]:
all_dists = []
min_dists = []
min_dists_z = []
min_dists_yx = []

for i, row in tqdm(enumerate(df.iter_rows(named = True))):
    tomo_id = row['tomo_id']
    motor_zyxas = row['motor_zyxA']
    if len(motor_zyxas) > 1:
        motor_zyxas = np.array(motor_zyxas)
        print(tomo_id, motor_zyxas.shape)
        dists = pdist(motor_zyxas, metric='euclidean')
        dists_z = pdist(motor_zyxas[:, 0:1], metric='euclidean')
        dists_y = pdist(motor_zyxas[:, 1:2], metric='euclidean')
        dists_x = pdist(motor_zyxas[:, 2:3], metric='euclidean')
        
        min_dist = dists.min()
        min_dists.append(min_dist)
        all_dists.extend(dists)
        min_dists_z.append(dists_z.min())
        if dists_z.min() == 0:
            print('SAME Z:', tomo_id)
        min_dist_yx = min(dists_y.min(), dists_x.min())
        if min_dist_yx == 0:
            print('SAME YX:', tomo_id)
        min_dists_yx.append(min_dist_yx)

print('ALL DISTS:', all_dists)
print('ALL MIN DISTS:', min_dists)
print('ALL MIN DISTS Z:', min_dists_z)
print('MIN DIST:', min([e for e in min_dists if e != 0]))
print('MIN DIST Z:', min([e for e in min_dists_z if e != 0]))
print('MIN DIST YX:', min(min_dists_yx))
        


648it [00:00, 66449.29it/s]

tomo_00e463 (6, 3)
SAME Z: tomo_00e463
tomo_08446f (2, 3)
tomo_0de3ee (2, 3)
tomo_1b82d1 (4, 3)
tomo_1da097 (2, 3)
tomo_226cd8 (10, 3)
SAME YX: tomo_226cd8
tomo_23c8a4 (2, 3)
tomo_2b3cdf (2, 3)
tomo_2daaee (4, 3)
tomo_37dd38 (3, 3)
tomo_381add (2, 3)
tomo_3a8480 (2, 3)
tomo_3e7407 (4, 3)
tomo_401341 (2, 3)
tomo_47ac94 (2, 3)
tomo_48dc93 (2, 3)
tomo_49725c (2, 3)
tomo_4e1b18 (3, 3)
tomo_507b7a (2, 3)
tomo_5b34b2 (2, 3)
tomo_5ba0cf (2, 3)
tomo_5d4c65 (4, 3)
tomo_634b06 (4, 3)
tomo_6521dc (3, 3)
tomo_67717e (2, 3)
tomo_6943e6 (2, 3)
tomo_6cf2df (2, 3)
tomo_754447 (2, 3)
tomo_7ca7c0 (3, 3)
tomo_8d2d48 (4, 3)
tomo_971966 (3, 3)
SAME Z: tomo_971966
SAME YX: tomo_971966
tomo_9722d1 (2, 3)
tomo_997240 (2, 3)
tomo_9a7701 (6, 3)
tomo_a549d6 (2, 3)
tomo_a5ac23 (2, 3)
tomo_a8073d (3, 3)
tomo_a84050 (2, 3)
tomo_a9d067 (3, 3)
tomo_ae347a (2, 3)
tomo_aec312 (2, 3)
tomo_aff073 (3, 3)
tomo_b0ded6 (2, 3)
tomo_b33d4e (2, 3)
tomo_b4a1f0 (4, 3)
SAME YX: tomo_b4a1f0
tomo_b7d014 (2, 3)
tomo_bad7b3 (2, 3)
tom

In [35]:
# SAME Z
df.filter(pl.col('tomo_id').is_in(['tomo_00e463', 'tomo_971966']))

tomo_id,Z,Y,X,voxel_spacing,ori_num_motors,num_motors,ZA,YA,XA,V,VA,motor_z,motor_y,motor_x,motor_zA,motor_yA,motor_xA,motor_zyx,motor_zyxA
str,i64,i64,i64,f64,i64,i64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,f64,object,object
"""tomo_00e463""",500,924,956,19.7,6,6,9850.0,18202.8,18833.2,441672000,3.3767e12,235.0,403.0,137.0,4629.5,7939.1,2698.9,"[[235.0, 403.0, 137.0], [243.0, 363.0, 153.0], [222.0, 379.0, 144.0], [225.0, 262.0, 628.0], [225.0, 241.0, 643.0], [231.0, 289.0, 632.0]]","[[4629.5, 7939.099999999999, 2698.9], [4787.099999999999, 7151.099999999999, 3014.1], [4373.4, 7466.3, 2836.7999999999997], [4432.5, 5161.4, 12371.6], [4432.5, 4747.7, 12667.1], [4550.7, 5693.3, 12450.4]]"
"""tomo_971966""",500,924,956,16.1,0,3,8050.0,14876.4,15391.6,441672000,1.8432e12,218.125,75.0,480.5,3511.8125,1207.5,7736.05,"[[218.125, 75.0, 480.5], [218.125, 838.5, 480.5], [242.0, 838.5, 496.5]]","[[3511.8125000000005, 1207.5, 7736.050000000001], [3511.8125000000005, 13499.85, 7736.050000000001], [3896.2000000000003, 13499.85, 7993.650000000001]]"


In [36]:
# SAME YX
df.filter(pl.col('tomo_id').is_in(['tomo_226cd8', 'tomo_971966', 'tomo_b4a1f0', 'tomo_da79d8']))

tomo_id,Z,Y,X,voxel_spacing,ori_num_motors,num_motors,ZA,YA,XA,V,VA,motor_z,motor_y,motor_x,motor_zA,motor_yA,motor_xA,motor_zyx,motor_zyxA
str,i64,i64,i64,f64,i64,i64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,f64,object,object
"""tomo_226cd8""",500,928,960,13.1,10,10,6550.0,12156.8,12576.0,445440000,1.0014e12,365.0,528.0,607.0,4781.5,6916.8,7951.7,"[[365.0, 528.0, 607.0], [312.0, 136.0, 99.0], [299.0, 227.0, 70.0], [332.0, 552.0, 58.0], [355.0, 656.0, 165.0], [366.0, 693.0, 269.0], [386.0, 693.0, 319.0], [388.0, 671.0, 433.0], [411.0, 318.0, 740.0], [354.0, 277.0, 756.0]]","[[4781.5, 6916.8, 7951.7], [4087.2, 1781.6, 1296.8999999999999], [3916.9, 2973.7, 917.0], [4349.2, 7231.2, 759.8], [4650.5, 8593.6, 2161.5], [4794.599999999999, 9078.3, 3523.9], [5056.599999999999, 9078.3, 4178.9], [5082.8, 8790.1, 5672.3], [5384.099999999999, 4165.8, 9694.0], [4637.4, 3628.7, 9903.6]]"
"""tomo_971966""",500,924,956,16.1,0,3,8050.0,14876.4,15391.6,441672000,1.8432e12,218.125,75.0,480.5,3511.8125,1207.5,7736.05,"[[218.125, 75.0, 480.5], [218.125, 838.5, 480.5], [242.0, 838.5, 496.5]]","[[3511.8125000000005, 1207.5, 7736.050000000001], [3511.8125000000005, 13499.85, 7736.050000000001], [3896.2000000000003, 13499.85, 7993.650000000001]]"
"""tomo_b4a1f0""",500,924,956,19.7,4,4,9850.0,18202.8,18833.2,441672000,3.3767e12,208.0,723.0,402.0,4097.6,14243.1,7919.4,"[[208.0, 723.0, 402.0], [226.0, 723.0, 373.0], [202.0, 716.0, 367.0], [223.0, 679.0, 886.0]]","[[4097.599999999999, 14243.1, 7919.4], [4452.2, 14243.1, 7348.099999999999], [3979.3999999999996, 14105.199999999999, 7229.9], [4393.099999999999, 13376.3, 17454.2]]"
"""tomo_da79d8""",500,924,956,19.7,4,4,9850.0,18202.8,18833.2,441672000,3.3767e12,239.0,186.0,492.0,4708.3,3664.2,9692.4,"[[239.0, 186.0, 492.0], [205.0, 184.0, 492.0], [223.0, 206.0, 491.0], [218.0, 165.0, 493.0]]","[[4708.3, 3664.2, 9692.4], [4038.5, 3624.7999999999997, 9692.4], [4393.099999999999, 4058.2, 9672.699999999999], [4294.599999999999, 3250.5, 9712.1]]"
